In [1]:
import pandas as pd
import tiktoken
from datasets import load_dataset
# from .autonotebook import tqdm as notebook_tqdm
from openai.embeddings_utils import get_embedding

c:\Users\techi\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [3]:
data_files = "arxiv-metadata-oai-snapshot.json"
pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
pubmed_dataset

Found cached dataset json (C:/Users/techi/.cache/huggingface/datasets/json/default-3c18e91474a8e8f0/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


Dataset({
    features: ['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'],
    num_rows: 2242698
})